# Chapter 4: Full Robot Simulatiom
Welcome to the fourth day of our hands-on course!
Today, you will focus on wirting the whole roboter plan and understand more of the basic concepts within pycram.
### Goal
By the end of the session, you will have successfully eecuted robot control programm.


# Getting Started
First we will load all the Libaries and spawn the Kitchen and Robot, as we did in earlier examples.

In [ ]:
from pycram.external_interfaces.ik import request_ik
from pycram.plan_failures import IKError
from pycram.ros.tf_broadcaster import TFBroadcaster
from pycram.ros.viz_marker_publisher import VizMarkerPublisher, AxisMarkerPublisher, CostmapPublisher
from pycram.utils import _apply_ik
from pycram.worlds.bullet_world import BulletWorld
from pycram.designators.action_designator import *
from pycram.designators.location_designator import *
from pycram.designators.object_designator import *
from pycram.datastructures.enums import ObjectType, WorldMode, TorsoState
from pycram.datastructures.pose import Pose, Transform
from pycram.process_module import simulated_robot, with_simulated_robot
from pycram.object_descriptors.urdf import ObjectDescription
from pycram.world_concepts.world_object import Object
from pycram.datastructures.dataclasses import Color

extension = ObjectDescription.get_file_extension()

world = BulletWorld(WorldMode.DIRECT)
world.allow_publish_debug_poses = True
viz = VizMarkerPublisher()
tf = TFBroadcaster()

robot_name = "pr2"
robot = Object(robot_name, ObjectType.ROBOT, f"{robot_name}{extension}", pose=Pose([1, 2, 0]))

apartment = Object("apartment", ObjectType.ENVIRONMENT, f"apartment-small{extension}")
milk = Object("milk", ObjectType.MILK, "milk.stl", pose=Pose([0.5, 2.5, 1]))
milk.color = Color(0, 0, 1, 1)
milk_desig = BelieveObject(names=["milk"])
robot_desig = BelieveObject(names=[robot_name])
apartment_desig = BelieveObject(names=["apartment"])


# Start we we left off with detecting the milk in the open fridge:

In [ ]:
with simulated_robot:
    poseHard = Pose([1.3, 2.7, 0], [0, 0, 1, 0])
    NavigateAction([poseHard]).resolve().perform()

    ParkArmsAction([Arms.BOTH]).resolve().perform()

    MoveTorsoAction([TorsoState.HIGH]).resolve().perform()
    handle_desig = ObjectPart(names=["handle_cab3_door_top"], part_of=apartment_desig.resolve())
    closed_location, opened_location = AccessingLocation(handle_desig=handle_desig.resolve(),
                                                         robot_desig=robot_desig.resolve()).resolve()
    OpenAction(object_designator_description=handle_desig, arms=[closed_location.arms[0]],
               start_goal_location=[closed_location, opened_location]).resolve().perform()
    
    LookAtAction(targets=[milk_desig.resolve().pose]).resolve().perform()
    obj_desig = DetectAction(milk_desig).resolve().perform()

Now what is left todo: Pick up the milk, Transport the milk, Place the milk close the fridge. This might sound for a human quite simple but for a robot controll programm this can be very heavy. Think about what knowledge youd need in order to be able to pick the milk. You will need knowledge on how to move your arm, where to move your arm how much foce youd need for in order to held the mmilk properly, is the milk full half full or not depending on that youd need different velocity / energy to pick it up. Is the milk open or closed, depending on that youd might need a different way to carry it to not spill anthing,  how to move to safely place the milk, where to place the milk and so on.
Within this tutorial we will not cover any possibilities and or limitation since our pycram world is a rapid fire simulation this means it is designed to have schemantic understanding of a robot controll programm like a cognitive architecture should, but to try out new plans the robot is moving faster then in a photrelsstic environemnt and we also teleport the robot instead of moving it along a path to safe time.

But what we will cover is: 
     1. Parking both arms for initial positioning.
      2. Determining the appropriate grasp based on object type.
      3. Finding a reachable pose for the robot to pick up the object with the designated arm.
       4. Navigating to the pick-up pose, performing the pick-up, and then parking arms again.
       5. Resolving a place location reachable by the robot to place the object.
       6. Navigating to the place location, performing the placement, and parking arms post-transport.

#### The Grasp
The Grasp it a tricky part in every robot controll programm. DEpending on where the perception system detected the object the object orientation can be very different. We want to be able to tell the robot " pick up the milk from front" or "from top" or other dirention. Natevily we want front top etc in a form that we human can use iteasily, for example nno matter how the milk is perceied in the fridge, the robot should always approach the object from front side of the fridge, if you rethink this to be on the other side of the kithn we have a orienttion problem. To overcome this axis problem we developed a calculate_object_face fucntion which takes an object as a parameter and then through rotation matri and where the robot stands calculates how the object is face dtowards the robot.
<details>

<summary>Click here for the math behind calculate_object:faces</summary>

```python
from pycram.designators.action_designator import DetectAction, LookAtAction, ParkArmsAction, NavigateAction
from pycram.designators.object_designator import BelieveObject
from pycram.datastructures.enums import Arms
from pycram.process_module import simulated_robot
from pycram.datastructures.pose import Pose

milk_desig = BelieveObject(names=["milk"])

with simulated_robot:
    ParkArmsAction([Arms.BOTH]).resolve().perform()

    NavigateAction([Pose([0, 1, 0], [0, 0, 0, 1])]).resolve().perform()

    LookAtAction(targets=[milk_desig.resolve().pose]).resolve().perform()

    obj_desig = DetectAction(milk_desig).resolve().perform()

    print(obj_desig)
```
</details>

we will be left off witha determined grasp by the faced rotation of the object
then we will use the CostmapLocation to find a place where the robot can stand to pick up the milk
#exercise: use the robot_desig the obj_desig an arm of your choice  and the detrmined grasp to find out a suitable pose with CostmapLocation

In [ ]:

## add your code here

<details>

<summary>Click here for the Solution.</summary>

```python
with simulated_robot:
   grasp = calculate_object_faces(self.object_designator)[0]

        pickup_loc = CostmapLocation(
            target=self.object_designator,
            reachable_for=robot_desig.resolve(),
            reachable_arm=self.arm,
            used_grasps=[grasp]
        )
```
</details>
You will be left off with a list of possible pick up location now we will iter over those poses in location to check if the pose is reachable for the arm.

´´´python
  pickup_pose = next((pose for pose in pickup_loc if self.arm in pose.reachable_arms), None)
        if not pickup_pose:
            raise ObjectUnfetchable(
                f"No reachable pose found for the robot to grasp the object: {self.object_designator} with arm: {self.arm}"
            )
 ´´´
 After that we Navigate to the found pose and to a PickUpAction. This pickupaaction takes the object designator the arm and the earlier determined grasp as parameters.
 

In [ ]:
## add your code here


<details>

<summary>Click here for the Solution.</summary>

```python
with simulated_robot:

    PickUpAction(spoon_desig, [Arms.LEFT], [Grasp.TOP]).resolve().perform()

```
</details>


Then we again use the Costmap in order to find a place to stand to place the objec ton a given pose. we hope you undertood how much information a robot needs to fullfill a basic pick and place order scenariou. We already put this all into a TransportAction which can be used by u.
Now your task is to write the full plan with trransporting in and closing the fridge. The milkplace pose is:     milk_target_pose = Pose([5.34, 3.55, 0.8])


In [ ]:
    milk_target_pose = Pose([5.34, 3.55, 0.8])


In [ ]:
## add your code here


<details>

<summary>Click here for the Solution.</summary>

```python
with simulated_robot:
    poseHard = Pose([1.3, 2.7, 0], [0, 0, 1, 0])
    NavigateAction([poseHard]).resolve().perform()

    ParkArmsAction([Arms.BOTH]).resolve().perform()

    MoveTorsoAction([TorsoState.HIGH]).resolve().perform()
    handle_desig = ObjectPart(names=["handle_cab3_door_top"], part_of=apartment_desig.resolve())
    closed_location, opened_location = AccessingLocation(handle_desig=handle_desig.resolve(),
                                                         robot_desig=robot_desig.resolve()).resolve()
    OpenAction(object_designator_description=handle_desig, arms=[closed_location.arms[0]],
               start_goal_location=[closed_location, opened_location]).resolve().perform()

    LookAtAction(targets=[milk_desig.resolve().pose]).resolve().perform()
    obj_desig = DetectAction(milk_desig).resolve().perform()
    milk_target_pose = Pose([5.34, 3.55, 0.8])
    ParkArmsAction([Arms.BOTH]).resolve().perform()
    MoveTorsoAction([TorsoState.HIGH]).resolve().perform()
    TransportAction(obj_desig, [Arms.LEFT], [milk_target_pose]).resolve().perform()
```
</details>


Now we have our first full transporting plan! Of course we are missing a lot of cognitive enabled stuff here but for the basic understanding this is really good already.
What is missing btu yet important?
For example failure handling. Failure handling nto only makes the robot more robust but it also makes us rethink but a plan of a robot controll system actual has to face. in order to design a genrelized failure handling ur controll programm must be semantic understood and open withut any hidden infomration to for example, retry, replan, transofrm, or react differnt to failures.

Another concept would be learning from memories. so we log everything that th robot did in the past and can replay revisit this to inform the robot abut what he is doing wrong this time, you can find more on this in our extra materials for chapter 6.


More physical udnerstanding
as described above the physical understanding is kinda cheated in this simulation, so how would be do this.

INSERT MULTIVERSE